## Import libraries

In [1]:
import tomopy
from helperFunctions import MoviePlotter
from tomoDataClass import tomoData

## Create Simulated Data

In [ ]:
#Set constants
numAngles = 200 #Number of projection angles
imageSize = 256 #Pixel size of one side of cubic array containing Shepp3D phantom

In [ ]:
#Create a simulated object
obj = tomopy.shepp3d(size=imageSize)
print(obj.shape)
print("Simulated Object")
MoviePlotter(obj) #Plots vertical slices through the phantom

In [ ]:
# Set projection angles
angles = tomopy.angles(nang=numAngles, ang1=1, ang2=360)

# create projection images/sinogram
projections = tomopy.project(obj, angles, pad=False)

print("Projections through object")
MoviePlotter(projections)

In [ ]:
tomo = tomoData(projections)
tomo.jitter(maxShift = 7)
tomo.add_noise()
print("Projections through Object with jitter and noise")
tomo.makeNotebookProjMovie()

## Align Data

In [ ]:
#Show bad reconstruction prior to alignment
tomo.reconstruct(algorithm='art')
print("\nBad reconstruction prior to alignment")
badRecon = tomo.recon.copy()
MoviePlotter(badRecon)

In [ ]:
tomo.reset_workingProjections(x_size=imageSize, y_size=imageSize) #You can adjust these for tighter cropping
tomo.normalize(isPhaseData=False)
tomo.cross_correlate_align(tolerance=0.1, max_iterations=15, stepRatio=1, yROI_Range=None, xROI_Range=None)
tomo.PMA(max_iterations=5, tolerance=0.05, algorithm='art', crop_bottom_center_y=imageSize, crop_bottom_center_x=imageSize, standardize=False)
tomo.make_updates_shift()

In [ ]:
tomo.makeNotebookProjMovie()

In [ ]:
tomo.reconstruct(algorithm='art')
print("\nGood Reconstruction after alignment")
tomo.makeNotebookReconMovie()